# When you have Multiple Access Points

Now lets discuss how to find a location when you have multiple access points. You can find this information in a couple of different places.

You can do this manually.  If a criminal accesses a website, opens an office document, or runs a java applet then you can collect this information from their computers with the following Windows Command.

"netsh wlan show networks mode=bssid | findstr "SSID Signal Channel"

This is the technique used by the "HoneyBadger" project.
https://github.com/lanmaster53/honeybadger

Setup can be a bit complicated so you might be more interested in his SAAS solution.
https://hb.lanmaster53.com/

**Speak with your legal team**

But this information also appears in the windows event logs when "Diagnose my network" buttons are clicked.  They are in event ID 6100.  Not all 6100 events have the wireless access points.



![alt text](images/Event6100.jpg "Event 6100")


Google provides an API that will provide a precise location when this information is available.  The first step is to build a data structure that google requires to resolve youre request.



In [ ]:
log_data = """
List of visible access point(s): 17 item(s) total, 17 item(s) displayed
        BSSID		BSS Type PHY	Signal(dB)	Chnl/freq    SSID
-------------------------------------------------------------------------
1E-8D-CB-84-FA-DF	Infra	 <unknown>	-62		5745000	 ARFF
06-8D-CB-84-FA-DF	Infra	 <unknown>	-63		5745000	 ARAwifi1
0A-8D-CB-84-FA-DF	Infra	 <unknown>	-62		5745000	 AirportPrivateWiFi
0E-8D-CB-84-FA-DF	Infra	 <unknown>	-63		5745000	 Airport_Wifi
02-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 (Unnamed Network)
1E-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 ARFF
06-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 ARAwifi1
0A-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 AirportPrivateWiFi
0C-8D-DB-84-FA-DF	Infra	 <unknown>	-42		11	 Airport_Wifi
F4-39-09-61-0E-1F	Infra	 <unknown>	-74		6	 DIRECT-1E-HP OfficeJet Pro 8710
B2-C1-9E-51-BE-D3	Infra	 <unknown>	-51		6	 ATT-WIFI-3191
FA-E4-F0-BE-E3-01	Infra	 <unknown>	-58		6	 Esneepez
54-75-D0-84-2E-F9	Infra	 g	-54		4	 Tailwinds Wi-Fi
54-75-D0-84-2E-F8	Infra	 g	-54		4	 (Unnamed Network)
5C-B0-66-D8-07-F6	Infra	 <unknown>	-57		1	 SBG6580-2-FD39B
6A-E7-C2-C5-D2-BD	Infra	 <unknown>	-60		1	 Samsung Galaxy_8515
0C-8C-24-6F-9E-28	Infra	 <unknown>	-84		10	 SNRS_6F9E28
"""


First we will use some regular expressions which are covered extensively in SEC573 to extract all of the BSSID and signal strengths from the record.


In [ ]:
import re
ap_data = re.findall(r"([\dA-F-]+)\s+?Infra.*?(-\d+)", log_data, re.MULTILINE)
ap_data


Next we build the JSON data structure that Googles API is expecting us to submit and we make the request.  Once we get back the record we extract the latitude and longitude and generate a google maps link.


In [ ]:
ap_list = []
for mac,signal in ap_data:
    ap_list.append({"macAddress":mac ,"signalStrength":signal})
google_data = {"considerIP": "false", "wifiAccessPoints": ap_list}
google_data



This data structure can now be submitted to googles API. But you have to have an API key.  To get an API key you have to provide google with a Credit Card. Rather than asking you to do this I'm going to give you my API key which is associated with my personal credit card. So please be kind and limit your request to those required for todays lab.


In [ ]:
import requests

#First borrow Mark Baggetts personal API key (please be kind)
#Change the IP to the one provided in class
key = requests.get("http://127.0.0.1:8000/key.txt").text

#Then generate the request
url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={key}'    
response = requests.post(url=url, json=google_data)    

#Then extract the data and print a google maps link
loc_record = response.json()
lat = loc_record.get("location").get("lat")
lng = loc_record.get("location").get("lng")
maps_url = f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"
print(loc_record.get("accuracy"))
print(maps_url)



Thats amazing.  It has an accuracy of approximatly 20 meters. But what if you are not lucky enough to find an event 6100. They are in fact rather rare to find on a forensics investigation.  What if rather than an event 6100 you are able to determine just two BSSIDs?   The google API will allow you to submit two BSSIDs with no signal strength.


In [ ]:
ap_data = [('0C-8C-24-6F-9E-28'), ('0C-8D-DB-84-FA-DF')]
ap_list = []
for mac in ap_data:
    ap_list.append({"macAddress":mac})
                                   
response = requests.post(url=url, json= {"considerIP": "false", "wifiAccessPoints": ap_list})    
loc_record = response.json()
lat = loc_record.get("location").get("lat")
lng = loc_record.get("location").get("lng")
maps_url = f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"
print(loc_record.get("accuracy"))
print(maps_url)



Our accuracy fell way down.  As a matter a fact, without the Signal strength it appears that google will always give you an accuracy of 150 meters.

Perhaps 150 meters is good enough. If not we can guess to force googles algorithms to give us a better location. Most of the time the signal strenght is somewhere between -40 and -90.  Anecdotially I believe the average is round -60. By setting an arbitrary signal strength of -60 we force google to choose the point that is equal distinace from the access points which is a better approximation than the generic response of 150 meters.

This time I will put this into a function that you can call.  The function requires that you pass it a list of BSSIDs and the google API key.  It returns back to you both the accuracy and a URL that you can use to see the location.



In [ ]:
def find_location(list_of_aps, key):
    url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={key}'       
    ap_list = []
    for mac in list_of_aps:
        ap_list.append({"macAddress":mac ,"signalStrength":"-60"})
    google_data = {"considerIP": "false", "wifiAccessPoints": ap_list}
    response = requests.post(url=url, json=google_data)    
    loc_record = response.json()
    lat = loc_record.get("location").get("lat")
    lng = loc_record.get("location").get("lng")
    accuracy = loc_record.get('accuracy')
    return accuracy, f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"

print("Here is a good response from google.")
ap_data = [('0C-8C-24-6F-9E-28'), ('0C-8D-DB-84-FA-DF')]
the_accuracy,the_url = find_location(ap_data, key)
print(f"The accuracy is {the_accuracy}")
print(the_url)

print("\n\nHere is a response from google when we send it made up BSSIDs.")
ap_data = [('11-22-33-44-55-66'), ('AA-BB-CC-DD-EE-FF')]
the_accuracy,the_url = find_location(ap_data, key)
print(f"The accuracy is {the_accuracy}")
print(the_url)




When google doesn't have any information about the Wireless access point pairs you provide it gives you a link to your current location based upon the IP address you are submitting it from. The accuracy will change depending upon your location. So don't use a high accuracy to detect when it doesn't have data.  Instead submit one request with made up BSSIDs and check to see if the responses match that.

Using this we can discover access point locations when google knows about any two of them that we find in history. Consider this scenario.  You find artificts on the laptop that indicate it was in the vicinity of 5 access points.   We will call them  AP1, AP2, AP3, AP4 and AP5.  Some of those APs may be in far off distant locations but other may be close enough to one another to establish a location.  

![alt text](images/aps.png "AP Locations")


### We ask google about locations.

We ask: Whats the location of AP1 and AP2?

Google: Here is a link to your current location (ie I don't know)

We ask: Whats the location of AP1 and AP3?

Google: Here is a link to your current location

We ask: Whats the location of AP1 and AP4?

Google: Google gives a location which by itself would be somewhere along the blue line.

We ask: Whats the location of AP1 and AP5?

Google: Here is a link to your current location

We ask: Whats the location of AP2 and AP3?

Google: Here is a link to your current location

We ask: Whats the locationa of AP2 and AP4?

Google: Google gives you a location which by itself would be somewhere along the green line. 

Now, by looking at the intersection of these sets we can identify that AP4 has a relationship with both AP2 and AP1. Then ...

We ask: What is the location of AP1, AP2, and AP4?

Google: Google gives us a location somewhere close to the purple square in the middle.

Repeating this process we can find locations that would otherwise be inaccessible through most APIs.

Python also makes building combinations of pretty simple by calling itertools.combinations.


In [ ]:
import itertools

list_of_aps = ['AP1','AP2','AP3','AP4','AP5']
list(itertools.combinations(list_of_aps,2))